# Unit 5 - NLP End Project - Customer Grievance Classification
## 2509 - Aravind Kothandaraman

## Streamlining the Customer Grievance Process

**Context:**
As a data scientist working for the front office of a major American multinational bank, you are responsible for enhancing customer service and ensuring compliance with financial regulations. Your current assignment involves analysing the customer complaints the bank has received over the past year.

**Problems:**
- The current time-consuming manual process for daily triaging and reviewing of customer complaints
- The complaints data is currently underutilized in enhancing the quality of products and services.

**Objective:**
The goal is to use NLP techniques, such as text classification and sentiment analysis, to efficiently gain insights into the underlying causes of customer grievances. By leveraging these methods, we aim to better understand and address customer grievances, ultimately improving our grievance redressal process.

## Step - 1: Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Step - 2: Load the Data

In [3]:
# Load the data from the Excel file
# it requires a few additional libraries to be installed in your environment
!pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]




[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
source_data = "banking_complaints_2023.xlsx"

# read the Excel file using pandas `ExcelFile` class.
# this will give us a list of sheet names in the excel file
xl = pd.ExcelFile(source_data)
xl.sheet_names

['complaints_banking_2023',
 'Department of Product',
 'Issues',
 'Data Dictionary']

In [6]:
# Read ALL sheets -> returns a dict: {sheet_name: DataFrame}
dfs_by_sheet = pd.read_excel(source_data, sheet_name=None)
{sheet: df.shape for sheet, df in dfs_by_sheet.items()}

{'complaints_banking_2023': (7011, 9),
 'Department of Product': (17, 2),
 'Issues': (2676, 2),
 'Data Dictionary': (12, 3)}

In [8]:
dfs_by_sheet["complaints_banking_2023"].head()

,Complaint ID,Date Received,Banking Product,Department,Issue ID,Complaint Description,State,ZIP,Bank Response
0,CID76118977,2023-01-01,Checking or savings account,CASA,I_3510635,on XX/XX/XX22 I opened a safe balance account ...,California,92311,Closed with monetary relief
1,CID98703933,2023-01-01,"Credit reporting, credit repair services, or o...",Credit Reports,I_3798538,There is an item from Bank of ABC on my credit...,California,91344,Closed with explanation
2,CID52036665,2023-01-01,Checking or savings account,CASA,I_3648593,On XX/XX/XX22 I found out that my account was ...,New York,10466,Closed with monetary relief
3,CID62581335,2023-01-01,Credit card or prepaid card,Credit Cards,I_6999080,I've had a credit card for years with Bank of ...,California,92127,Closed with monetary relief
4,CID65731164,2023-01-01,Checking or savings account,CASA,I_3648593,This issue has to do with the way that Bank of...,New Jersey,7946,Closed with explanation


In [ ]:
# Read SELECTED sheets by name
# selected_sheets = dfs_by_sheet.keys()  # replace with e.g. ["Sheet1", "Sheet2"]
# dfs_selected = pd.read_excel(source_data, sheet_name=list(selected_sheets))
# {sheet: df.shape for sheet, df in dfs_selected.items()}

In [10]:
complaints_df = dfs_by_sheet["complaints_banking_2023"]
complaints_df.shape

(7011, 9)

In [11]:
complaints_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7011 entries, 0 to 7010
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Complaint ID           7011 non-null   object        
 1   Date Received          7011 non-null   datetime64[ns]
 2   Banking Product        7011 non-null   object        
 3   Department             7011 non-null   object        
 4   Issue ID               7011 non-null   object        
 5   Complaint Description  7011 non-null   object        
 6   State                  6984 non-null   object        
 7   ZIP                    6981 non-null   object        
 8   Bank Response          7011 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 493.1+ KB


## Step - 3: Data Preprocessing

In [13]:
# check for missing values
complaints_df.isnull().sum()

Complaint ID              0
Date Received             0
Banking Product           0
Department                0
Issue ID                  0
Complaint Description     0
State                    27
ZIP                      30
Bank Response             0
dtype: int64

### Check for data types and convert if necessary

In [14]:
# rename columns for easier access
complaints_df = complaints_df.rename(columns={
    "Complaint ID": "complaint_id",
    "Date Received": "date_received",
    "Banking Product": "product",
    "Department": "department",
    "Issue ID": "issue_id",
    "Complaint Description": "complaint_description",
    "State": "state",
    "ZIP": "zip",
    "Bank Response": "bank_response",
})

In [17]:
print(complaints_df.head())

complaints_df.info()

  complaint_id date_received  \
0  CID76118977    2023-01-01   
1  CID98703933    2023-01-01   
2  CID52036665    2023-01-01   
3  CID62581335    2023-01-01   
4  CID65731164    2023-01-01   

                                             product      department  \
0                        Checking or savings account            CASA   
1  Credit reporting, credit repair services, or o...  Credit Reports   
2                        Checking or savings account            CASA   
3                        Credit card or prepaid card    Credit Cards   
4                        Checking or savings account            CASA   

    issue_id                              complaint_description       state  \
0  I_3510635  on XX/XX/XX22 I opened a safe balance account ...  California   
1  I_3798538  There is an item from Bank of ABC on my credit...  California   
2  I_3648593  On XX/XX/XX22 I found out that my account was ...    New York   
3  I_6999080  I've had a credit card for years with Bank o

In [29]:
# change the data types for the key columns
complaints_df = complaints_df.astype({
    "complaint_id": "string",
    "product": "string",
    "issue_id": "string",
    "department": "string",
    "complaint_description": "string",
    "state": "string",
    "zip": "string",
    "bank_response": "string",
    "date_received": "datetime64[ns]",
})

In [30]:
complaints_df.dtypes

complaint_id             string[python]
date_received            datetime64[ns]
product                  string[python]
department               string[python]
issue_id                 string[python]
complaint_description    string[python]
state                    string[python]
zip                      string[python]
bank_response            string[python]
dtype: object

In [31]:
complaints_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7011 entries, 0 to 7010
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   complaint_id           7011 non-null   string        
 1   date_received          7011 non-null   datetime64[ns]
 2   product                7011 non-null   string        
 3   department             7011 non-null   string        
 4   issue_id               7011 non-null   string        
 5   complaint_description  7011 non-null   string        
 6   state                  7011 non-null   string        
 7   zip                    7011 non-null   string        
 8   bank_response          7011 non-null   string        
dtypes: datetime64[ns](1), string(8)
memory usage: 493.1 KB


### Identify date ranges

In [33]:
# identify the date range of complaints
min_date = complaints_df["date_received"].min()
max_date = complaints_df["date_received"].max()
print(f"Date range: {min_date} to {max_date}")

# duh... the file name already says 2023 :)
print("duh..! the file name already says 2023 :)")

Date range: 2023-01-01 00:00:00 to 2023-10-21 00:00:00
duh..! the file name already says 2023 :)


### Create a function for Preprocessing Text Data

In [34]:
# create a function for text preprocessing

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("wordnet")

def preprocess_text(text):
    # convert to lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # remove numbers using regex is easier and ideal
    text = re.sub(r"\d+", "", text)
    # remove extra whitespace
    text = re.sub(r"\s+", " ", text).strip()
    # tokenize the text
    tokens = text.split()
    # remove stopwords
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    # applying lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # join tokens back to string
    preprocessed_text = " ".join(tokens)
    return preprocessed_text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aravi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aravi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
# quick test the function
sample_text = complaints_df["complaint_description"].iloc[0]
print(sample_text)
print("-"*50)
print(preprocess_text(sample_text))

on XX/XX/XX22 I opened a safe balance account online using my payroll check of over {$600.00} as my account opener. On XX/XX/XX22, I attempted to login to my app and received a " account locked contact customer service @ XXXX '' error message. I placed the call and was told I would have to go to a branch to verify my identity, which I did. At the same time, I was told I had a positive balance of of {$650.00} and could make a withdrawal from the teller, which I did for {$400.00} leaving a balance of + {$250.00}. Shortly after that, I deposited a mobile check for {$5XX.00}. On XX/XX/XX22, again I attempted to login and received the previous error message along with 3 emails stating a hold was placed on my account. Contacting the same previous number I was told that my acct was being closed by the Risk Dept. due to lack of identity verification, which I told them I had done the day before. I was then told it didn't matter the account was already being closed and it was not reversible, so 

The function `preprocess_text` performs the following steps:
1. Converts text to lowercase
2. Removes punctuation
3. Removes numbers using regex
4. Removes extra whitespace
5. Tokenizes the text into words
6. Removes stopwords
7. Applies lemmatization to reduce words to their base form
8. Joins the tokens back into a single string

And it does what it is supposed to do!

### Apply the function to the complaint descriptions

In [37]:
# apply the preprocessing function to the complaint descriptions and save the result in a new column
complaints_df["preprocessed_description"] = complaints_df["complaint_description"].apply(preprocess_text)

In [44]:
# lets see the results along with their original lengths
complaints_df[["complaint_description", "preprocessed_description"]].assign(
    complaint_description_len=complaints_df["complaint_description"].str.len(),
    preprocessed_description_len=complaints_df["preprocessed_description"].str.len(),
).head()

# len(complaints_df["complaint_description"]),
# len(complaints_df["preprocessed_description"])


,complaint_description,preprocessed_description,complaint_description_len,preprocessed_description_len
0,on XX/XX/XX22 I opened a safe balance account ...,xxxxxx opened safe balance account online usin...,1827,1035
1,There is an item from Bank of ABC on my credit...,item bank abc credit report belong must remove...,354,189
2,On XX/XX/XX22 I found out that my account was ...,xxxxxx found account frozen apparent reason we...,426,258
3,I've had a credit card for years with Bank of ...,ive credit card year bank abc xxxxxxxx paid ba...,2153,1280
4,This issue has to do with the way that Bank of...,issue way bank abc account linking bill pay pa...,1339,869


## Step - 4: Text Feature Engineering